In [8]:
!pip install pennylane

     |████████████████████████████████| 409kB 8.1MB/s 
  Created wheel for pennylane: filename=PennyLane-0.14.1-cp37-none-any.whl size=481980 sha256=8fcd82734d85023e03a18960c42541fa5eb419889ef142f38d6f3b50fd20a9e9
  Stored in directory: /root/.cache/pip/wheels/6e/a9/c9/04941d6dd58b3c111cbd1389bf63ba1b23362c137359af4626
Successfully built pennylane


In [ ]:
import keras

In [ ]:
# Sentiment prediction

from keras.datasets import imdb
from keras import preprocessing
from pennylane import numpy as np

cor_size = 16
max_len = 3

(xtrain, ytrain), (xtest, ytest) = imdb.load_data(num_words=cor_size)
xtrain = preprocessing.sequence.pad_sequences(xtrain, maxlen=max_len)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
x_train = np.zeros((len(xtrain), 4*max_len))
for i, x in enumerate(xtrain):
  temp = []
  for y in x:
    
    temp += [int(z) for z in bin(y)[2:].zfill(4)]

  x_train[i] = np.array(temp)


In [ ]:
import pennylane as qml
from pennylane.templates import QAOAEmbedding
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=12)

@qml.qnode(dev)
def circuit(weights, x=None):
  QAOAEmbedding(features=x, weights=weights, wires=range(12))
  return qml.expval(qml.PauliZ(0))

In [ ]:
def cost(weights, X=None, y=None):
  predictions = [circuit(weights=weights, x=x) for x in X]
  
  loss = 0
  for i in range(len(predictions)):
    # v = np.inner(predictions[i], y[i])
    # if v==0:
    #   v=0.0001
    loss += (predictions[i]-y[i])**2
  
  return loss/(len(predictions))

In [ ]:
X = x_train[:100]
y = 2*ytrain[:100]-1
weights = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=(4, 24))
opt = qml.MomentumOptimizer()
n_it = 100
loss = []
for i in range(n_it):
  weights, ls = opt.step_and_cost(lambda w: cost(weights=w, X=X, y=y), weights)
  loss.append(ls)
loss

[1.0402421518769924,
 1.037450543378709,
 1.0323847219237516,
 1.025695265303702,
 1.0180932423290456,
 1.0102807230759767,
 1.0028860787477196,
 0.9964091809898271,
 0.991183194455509,
 0.9873584905946968,
 0.9849106501355505,
 0.9836699027451752,
 0.9833653668597264,
 0.9836753821122963,
 0.9842754553170485,
 0.9848773725292748,
 0.9852558998756652,
 0.9852622700526658,
 0.9848257443191026,
 0.9839457752109569,
 0.982677784637585,
 0.9811155427432923,
 0.9793727987822937,
 0.9775663213418642,
 0.9758019313478745,
 0.9741645022979611,
 0.972712297579732,
 0.9714754623426346,
 0.9704580396871725,
 0.9696425813748069,
 0.9689962925006407,
 0.968477679154979,
 0.9680428239896536,
 0.9676506479585203,
 0.9672667766647334,
 0.966865874751698,
 0.9664325138248835,
 0.9659607853243395,
 0.965452958109957,
 0.9649175172650518,
 0.9643669153704683,
 0.9638153308172319,
 0.963276669774493,
 0.9627629785016716,
 0.962283359130445,
 0.9618434121651813,
 0.9614451688330683,
 0.9610874305267342,
 0

In [ ]:
predictions = [circuit(weights=weights, x=x) for x in X]
predictions = [np.sign(p) for p in predictions]
training_accuracy = list(predictions*y).count(1)/len(y)
training_accuracy

0.59

In [ ]:
xtest = preprocessing.sequence.pad_sequences(xtest, maxlen=max_len)
x_test = np.zeros((len(xtest), 4*max_len))
for i, x in enumerate(xtest):
  temp = []
  for y in x:
    # print(bin(y)[2:].zfill(7))
    # x_train[i] = []
    temp += [int(z) for z in bin(y)[2:].zfill(4)]
    # x_train[i] = [int(z) for z in bin(y)[2:].zfill(7)]
  x_test[i] = np.array(temp) 

xt = x_test[:25]
yt = 2*ytest[:25]-1
predictions = [circuit(weights=weights, x=x) for x in xt]
predictions = [np.sign(p) for p in predictions]
accuracy = list(predictions*yt).count(1)/len(yt)
print(accuracy)

0.44


In [1]:
# Auto-completer
import pennylane as qml
from pennylane.templates import QAOAEmbedding
from pennylane import numpy as np
import string 

corpus = "Same old dive, same old end of the work week drink Bartender knows my name, but I don't mind She kicks 'em up strong, serves me up right And here I go again I'm drinkin' one, I'm drinkin' two I got my heartache medication, a strong dedication To gettin' over you, turnin' me loose On that hardwood jukebox lost in neon time My heartache medication, well it suits me fine And I'm drinkin' enough to take you off my mindI got my heartache medication"
table = str.maketrans(dict.fromkeys(string.punctuation))
corpus = corpus.translate(table)
tokens = list(corpus.lower().split())
w = 2
print(len(tokens))
vocab, index = {}, 1 # start indexing from 1
reverse_vocab = {}
vocab['<pad>'] = 0 # add a padding token 
for token in tokens:
  if token not in vocab: 
    vocab[token] = index
    index += 1

for item in vocab.keys():
  reverse_vocab[vocab[item]] = item

vocab_size = len(vocab)
print(vocab)

X = []
y = []
for i in range(w, len(tokens)):
  temp1 = []
  temp2 = np.array([0]*2**7)
  for j in range(1,w+1):
    # index.append(bin(vocab[corpus[i-j]])[2:])
    temp1 += [int(z) for z in bin(vocab[tokens[i-j]])[2:].zfill(7)]
  temp2[vocab[tokens[i]]] = 1
  X.append(temp1)
  y.append(temp2)

X = np.array(X)
y = np.array(y)


84
{'<pad>': 0, 'same': 1, 'old': 2, 'dive': 3, 'end': 4, 'of': 5, 'the': 6, 'work': 7, 'week': 8, 'drink': 9, 'bartender': 10, 'knows': 11, 'my': 12, 'name': 13, 'but': 14, 'i': 15, 'dont': 16, 'mind': 17, 'she': 18, 'kicks': 19, 'em': 20, 'up': 21, 'strong': 22, 'serves': 23, 'me': 24, 'right': 25, 'and': 26, 'here': 27, 'go': 28, 'again': 29, 'im': 30, 'drinkin': 31, 'one': 32, 'two': 33, 'got': 34, 'heartache': 35, 'medication': 36, 'a': 37, 'dedication': 38, 'to': 39, 'gettin': 40, 'over': 41, 'you': 42, 'turnin': 43, 'loose': 44, 'on': 45, 'that': 46, 'hardwood': 47, 'jukebox': 48, 'lost': 49, 'in': 50, 'neon': 51, 'time': 52, 'well': 53, 'it': 54, 'suits': 55, 'fine': 56, 'enough': 57, 'take': 58, 'off': 59, 'mindi': 60}


In [2]:
dev = qml.device("default.qubit", wires=14)

@qml.qnode(dev)
def circuit(weights, x=None):
  QAOAEmbedding(features=x, weights=weights, wires=range(14))
  return qml.probs(wires=range(7))

In [3]:
def cost(weights, X=None, y=None):
  predictions = [circuit(weights=weights, x=x) for x in X]
  
  loss = 0
  for i in range(len(predictions)):
    v = np.inner(predictions[i], y[i])
    # if v==0:
    #   v=0.0001
    loss -= v
  
  return loss


In [20]:
# weights = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=(4, 28))
opt = qml.MomentumOptimizer()
n_it = 10
loss = []
for i in range(n_it):
  weights, ls = opt.step_and_cost(lambda w: cost(weights=w, X=X, y=y), weights)
  loss.append(ls)
loss

[-31.197485094847583,
 -31.261759697959462,
 -31.366847806003523,
 -31.493335549709826,
 -31.63726835244995,
 -31.8027456960928,
 -31.989154248567292,
 -32.18931506642419,
 -32.39744951647307,
 -32.6140496230716]

In [21]:
loss

[-31.197485094847583,
 -31.261759697959462,
 -31.366847806003523,
 -31.493335549709826,
 -31.63726835244995,
 -31.8027456960928,
 -31.989154248567292,
 -32.18931506642419,
 -32.39744951647307,
 -32.6140496230716]

In [22]:
def func(x):
  index = x[0:7]
  first_index = 0
  for i, j in enumerate(index):
    first_index += 2**(6-i)*j
  
  index = x[7:]
  second_index = 0
  for i, j in enumerate(index):
    second_index += 2**(6-i)*j
  
  return first_index, second_index

def accuracy(predictions, labels):
  return 1/len(predictions)*[p==l for p,l in zip(predictions, labels)].count(True) 


print(corpus)
predicted_words = []
labels = []
similarity = []
for i, context in enumerate(X):
  prediction = circuit(weights, x=context)
  index = np.argmax(prediction)
  # predicted_words.append(reverse_vocab[index])
  # print(reverse_vocab())
  first_index, second_index = func(context)
  # print("{}".format())
  # print(reverse_vocab[int(second_index)], reverse_vocab[int(first_index)])
  if index not in reverse_vocab.keys():
      print("no prediction!")
  else:
    predicted_words.append(reverse_vocab[index])
    labels.append(reverse_vocab[np.argmax(y[i])])
    similarity.append(np.inner(prediction, y[i]))
    a, b, c, d = reverse_vocab[int(second_index)], reverse_vocab[int(first_index)], reverse_vocab[index],  reverse_vocab[np.argmax(y[i])]
    print("{}  {}, {} | {}".format(a, b, c, d))
    # print(reverse_vocab[index], reverse_vocab[np.argmax(y[i])])

accuracy(predicted_words, labels)


Same old dive same old end of the work week drink Bartender knows my name but I dont mind She kicks em up strong serves me up right And here I go again Im drinkin one Im drinkin two I got my heartache medication a strong dedication To gettin over you turnin me loose On that hardwood jukebox lost in neon time My heartache medication well it suits me fine And Im drinkin enough to take you off my mindI got my heartache medication
same  old, dive | dive
old  dive, <pad> | same
dive  same, old | old
same  old, dive | end
old  end, of | of
end  of, the | the
of  the, work | work
the  work, end | week
work  week, drink | drink
week  drink, bartender | bartender
drink  bartender, knows | knows
bartender  knows, week | my
knows  my, name | name
my  name, but | but
name  but, i | i
but  i, my | dont
i  dont, mind | mind
dont  mind, she | she
mind  she, kicks | kicks
she  kicks, dont | em
kicks  em, up | up
em  up, strong | strong
up  strong, serves | serves
strong  serves, em | me
serves  me, ri

0.4567901234567901

In [48]:
# import numpy as np
ten_per_cent_vocab = list(vocab.keys())[1:21]

import random

def generate_pairs(tpv):
  pairs = []
  for i, word1 in enumerate(tpv):
    for word2 in tpv[i+1:]:
      pairs.append((word1, word2))
      # pairs.append((word2, word1))
  return pairs

def generate_features(pair):
  # for pair in pairs:
  temp = []
  word1, word2 = pair[0], pair[1]
  temp += [int(z) for z in bin(vocab[word2])[2:].zfill(7)]
  temp += [int(z) for z in bin(vocab[word1])[2:].zfill(7)]
  
  return np.array(temp)

def generate_lyrics(pairs, window):
  lyrics = {}
  for i, pair in enumerate(pairs):
    if i>8:
      break
    x = generate_features(pair)
    temp = pair[1]
    # print(x, temp)
    lyrics[i] = [pair[0], pair[1]]
    value_window = random.choice(window)
    for j in range(value_window):
      prediction = circuit(weights, x=x)
      index = np.argmax(prediction)
      if index not in reverse_vocab.keys():
        print("no prediction!")
        break
      else:
        predicted_word = reverse_vocab[index]
        # print(predicted_word)
        lyrics[i].append(predicted_word)
        x = generate_features((temp, predicted_word))
        temp = predicted_word
  
  return lyrics

In [55]:
from random import shuffle

pairs = generate_pairs(ten_per_cent_vocab)
shuffle(pairs)
lyrics = generate_lyrics(pairs, [2,3])
for i in range(8):
  verse = " ".join(lyrics[i]).capitalize()
  if i==4:
    print("")
  print(verse)

Work bartender knows week
Same mind she kicks dont
Same work end of
Dive name but i my

The bartender knows week drink
My she kicks dont mind
Dive but i my name
The she kicks dont
